In [21]:
from tqdm import tqdm_notebook as tqdm
import preprocessor as p
import numpy as np

In [22]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.datasets as dsets
from torch.autograd import Variable
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
import torch.optim as optim
import torch.nn.functional as F

In [23]:
from transformers import *

In [24]:
from collections import Counter
import spacy
from tqdm import tqdm, tqdm_notebook, tnrange
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, confusion_matrix
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler


In [25]:
import pickle as pkl
import matplotlib.pyplot as plt
import random
import json

In [26]:
dataset = 'twitter15'
    
if dataset == 'twitter15':
    %run ../twitter15/twitter15_text_processing.ipynb
    with open('./kfolds_twitter15.pickle','rb') as f:
        kfold_df = pkl.load(f)
    datadir = './twitter15_kfold_results/'
    splitnum =  1117
    
if dataset == 'twitter16':
    %run ../twitter16/twitter16_text_processing.ipynb
    with open('./kfolds_twitter16.pickle','rb') as f:
        kfold_df = pkl.load(f)
    datadir = './twitter16_kfold_results/'
    splitnum =  613

In [27]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

I0307 22:52:50.301383 140645836187392 tokenization_utils.py:374] loading file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-vocab.txt from cache at /home/nikhil.pinnaparaju/.cache/torch/transformers/26bc1ad6c0ac742e9b52263248f6d0f00068293b33709fae12320c0e35ccfbbb.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084


In [28]:
def pad_data(s, maxlen=30):
        padded = np.zeros((maxlen,), dtype=np.int64)
        if len(s) > maxlen: padded[:] = s[:maxlen]
        else: padded[:len(s)] = s
        return padded

In [29]:
def trainModelWithKfold(foldnum,datadir):
    modelname = 'bertTextEnc'
    labels = {'true':0,'false':1,'unverified':2,'non-rumor':3}
    
    train_ids, valid_ids = kfold_df[foldnum]
    
    dataids = list(twitter15_text.keys())
    X = [ twitter15_text[x] for x in train_ids ] + [ twitter15_text[x] for x in valid_ids ]
    y = [ labels[twitter15_labels[x]] for x in train_ids] + [ labels[twitter15_labels[x]] for x in valid_ids]
    sent2idx = [torch.tensor(tokenizer.encode(x, add_special_tokens=True)) for x in X]
    padded = [ pad_data(x) for x in sent2idx ]
    
    attention_masks = []

    # For each sentence...
    for sent in padded:

        # Create the attention mask.
        #   - If a token ID is 0, then it's padding, set the mask to 0.
        #   - If a token ID is > 0, then it's a real token, set the mask to 1.
        att_mask = [int(token_id > 0) for token_id in sent]

        # Store the attention mask for this sentence.
        attention_masks.append(att_mask)
        
    train_inputs, validation_inputs, train_labels, validation_labels = padded[:splitnum],padded[splitnum:],y[:splitnum],y[splitnum:]
    # Do the same for the masks.
    train_masks, validation_masks = attention_masks[:splitnum],attention_masks[splitnum:]

    train_inputs = torch.tensor(train_inputs)
    validation_inputs = torch.tensor(validation_inputs)

    train_labels = torch.tensor(train_labels)
    validation_labels = torch.tensor(validation_labels)

    train_masks = torch.tensor(train_masks)
    validation_masks = torch.tensor(validation_masks)
    
    batch_size = 32

    # Create the DataLoader for our training set.
    train_data = TensorDataset(train_inputs, train_masks, train_labels)
    train_sampler = RandomSampler(train_data)
    train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

    # Create the DataLoader for our validation set.
    validation_data = TensorDataset(validation_inputs, validation_masks, validation_labels)
    validation_sampler = SequentialSampler(validation_data)
    validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=batch_size)
    
    if torch.cuda.is_available():
        device = 'cuda:1'
    else:
        device = 'cpu'
        
    model = BertForSequenceClassification.from_pretrained(
        "bert-base-uncased", # Use the 12-layer BERT model, with an uncased vocab.
        num_labels = 4, # The number of output labels--2 for binary classification.
                        # You can increase this for multi-class tasks.   
    ).to(device)
    
    optimizer = AdamW(model.parameters(),
                  lr = 2e-5, # args.learning_rate - default is 5e-5, our notebook had 2e-5
                  eps = 1e-8 # args.adam_epsilon  - default is 1e-8.
                )
    
    # Number of training epochs (authors recommend between 2 and 4)
    epochs = 8 

    # Total number of training steps is number of batches * number of epochs.
    total_steps = len(train_dataloader) * epochs
    
    # This training code is based on the `run_glue.py` script here:
    # https://github.com/huggingface/transformers/blob/5bfcd0485ece086ebcbed2d008813037968a9e58/examples/run_glue.py#L128


    # Set the seed value all over the place to make this reproducible.
    seed_val = 2018

    random.seed(seed_val)
    np.random.seed(seed_val)
    torch.manual_seed(seed_val)
    torch.cuda.manual_seed_all(seed_val)

    # Store the average loss after each epoch so we can plot them.
    loss_values = []
    maxAcc = 0
    bestcr = 0
    

    # For each epoch...
    for epoch_i in range(0, epochs):

        # ========================================
        #               Training
        # ========================================

        # Perform one full pass over the training set.

        print("")
        print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
        print('Training...')

        # Measure how long the training epoch takes.
        t0 = time.time()

        # Reset the total loss for this epoch.
        total_loss = 0

        # Put the model into training mode. Don't be mislead--the call to 
        # `train` just changes the *mode*, it doesn't *perform* the training.
        # `dropout` and `batchnorm` layers behave differently during training
        # vs. test (source: https://stackoverflow.com/questions/51433378/what-does-model-train-do-in-pytorch)
        model.train()

        # For each batch of training data...
        for step, batch in enumerate(train_dataloader):

            # Progress update every 40 batches.
            if step % 40 == 0 and not step == 0:
                # Calculate elapsed time in minutes.
                elapsed = format_time(time.time() - t0)

                # Report progress.
                print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))

            # Unpack this training batch from our dataloader. 
            #
            # As we unpack the batch, we'll also copy each tensor to the GPU using the 
            # `to` method.
            #
            # `batch` contains three pytorch tensors:
            #   [0]: input ids 
            #   [1]: attention masks
            #   [2]: labels 
            b_input_ids = batch[0].to(device)
            b_input_mask = batch[1].to(device)
            b_labels = batch[2].to(device)

            # Always clear any previously calculated gradients before performing a
            # backward pass. PyTorch doesn't do this automatically because 
            # accumulating the gradients is "convenient while training RNNs". 
            # (source: https://stackoverflow.com/questions/48001598/why-do-we-need-to-call-zero-grad-in-pytorch)
            model.zero_grad()        

            # Perform a forward pass (evaluate the model on this training batch).
            # This will return the loss (rather than the model output) because we
            # have provided the `labels`.
            # The documentation for this `model` function is here: 
            # https://huggingface.co/transformers/v2.2.0/model_doc/bert.html#transformers.BertForSequenceClassification
            outputs = model(b_input_ids, 
                        token_type_ids=None, 
                        attention_mask=b_input_mask, 
                        labels=b_labels)

            # The call to `model` always returns a tuple, so we need to pull the 
            # loss value out of the tuple.
            loss = outputs[0]

            # Accumulate the training loss over all of the batches so that we can
            # calculate the average loss at the end. `loss` is a Tensor containing a
            # single value; the `.item()` function just returns the Python value 
            # from the tensor.
            total_loss += loss.item()

            # Perform a backward pass to calculate the gradients.
            loss.backward()

            # Clip the norm of the gradients to 1.0.
            # This is to help prevent the "exploding gradients" problem.
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

            # Update parameters and take a step using the computed gradient.
            # The optimizer dictates the "update rule"--how the parameters are
            # modified based on their gradients, the learning rate, etc.
            optimizer.step()

            # Update the learning rate.

        # Calculate the average loss over the training data.
        avg_train_loss = total_loss / len(train_dataloader)            

        # Store the loss value for plotting the learning curve.
        loss_values.append(avg_train_loss)

        print("")
        print("  Average training loss: {0:.2f}".format(avg_train_loss))
        print("  Training epcoh took: {:}".format(format_time(time.time() - t0)))

        # ========================================
        #               Validation
        # ========================================
        # After the completion of each training epoch, measure our performance on
        # our validation set.

        print("")
        print("Running Validation...")

        t0 = time.time()

        # Put the model in evaluation mode--the dropout layers behave differently
        # during evaluation.
        model.eval()

        # Tracking variables 
        allLabels = []
        allPreds = []
        eval_loss, eval_accuracy = 0, 0
        nb_eval_steps, nb_eval_examples = 0, 0

        # Evaluate data for one epoch
        for batch in validation_dataloader:

            # Add batch to GPU
            batch = tuple(t.to(device) for t in batch)

            # Unpack the inputs from our dataloader
            b_input_ids, b_input_mask, b_labels = batch

            # Telling the model not to compute or store gradients, saving memory and
            # speeding up validation
            with torch.no_grad():        

                # Forward pass, calculate logit predictions.
                # This will return the logits rather than the loss because we have
                # not provided labels.
                # token_type_ids is the same as the "segment ids", which 
                # differentiates sentence 1 and 2 in 2-sentence tasks.
                # The documentation for this `model` function is here: 
                # https://huggingface.co/transformers/v2.2.0/model_doc/bert.html#transformers.BertForSequenceClassification
                outputs = model(b_input_ids, 
                                token_type_ids=None, 
                                attention_mask=b_input_mask)

            # Get the "logits" output by the model. The "logits" are the output
            # values prior to applying an activation function like the softmax.
            logits = outputs[0]

            # Move logits and labels to CPU
            logits = logits.detach().cpu().numpy()
            label_ids = b_labels.to('cpu').numpy()
            allPreds += torch.max(torch.tensor(logits),1)[1].numpy().tolist()
            allLabels += label_ids.tolist()
    #         print(logits)

            # Calculate the accuracy for this batch of test sentences.
            tmp_eval_accuracy = flat_accuracy(logits, label_ids)

            # Accumulate the total accuracy.
            eval_accuracy += tmp_eval_accuracy

            # Track the number of batches
            nb_eval_steps += 1

        valacc = accuracy_score(allLabels, allPreds)
        recscore = recall_score(allLabels, allPreds,average='macro')
        precscore = precision_score(allLabels, allPreds,average='macro')
        f1score = f1_score(allLabels, allPreds,average='macro')
        cr = classification_report(allLabels, allPreds,output_dict=True)

        cr['acc'] = valacc
        cr['fold'] = foldnum
        cr['modelname'] = modelname
        
        if valacc > maxAcc:
            maxAcc = valacc
            bestcr = cr
            torch.save({'state_dict': model.bert.state_dict()}, datadir+modelname+'_'+str(foldnum)+'.pth')

        print(f'acc: {valacc} recall {recscore} prec: {precscore} f1: {f1score}')

        print("  Accuracy: {0:.2f}".format(eval_accuracy/nb_eval_steps))
        print("  Validation took: {:}".format(format_time(time.time() - t0)))

    print("Training complete!")
    with open(datadir+cr['modelname']+'.json', 'a') as fp:
        json.dump(bestcr, fp)
        fp.write('\n')
        
    return bestcr

In [30]:
# Function to calculate the accuracy of our predictions vs labels
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [31]:
import time
import datetime
import random

def format_time(elapsed):
    '''
    Takes a time in seconds and returns a string hh:mm:ss
    '''
    # Round to the nearest second.
    elapsed_rounded = int(round((elapsed)))
    
    # Format as hh:mm:ss
    return str(datetime.timedelta(seconds=elapsed_rounded))

In [32]:
for i in range(0,4):
    trainModelWithKfold(i,datadir)

I0307 22:52:52.391630 140645836187392 configuration_utils.py:151] loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-config.json from cache at /home/nikhil.pinnaparaju/.cache/torch/transformers/4dad0251492946e18ac39290fcfe91b89d370fee250efe9521476438fe8ca185.8f56353af4a709bf5ff0fbc915d8f5b42bfff892cbb6ac98c3c45f481a03c685
I0307 22:52:52.393509 140645836187392 configuration_utils.py:168] Model config {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "finetuning_task": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "num_labels": 4,
  "output_attentions": false,
  "output_hidden_states": false,
  "output_past": true,
  "pruned_heads": {},
  "torchscript": false,
  "type_vocab_size": 2,
  "use_bf


======== Epoch 1 / 8 ========
Training...

  Average training loss: 1.38
  Training epcoh took: 0:00:08

Running Validation...
acc: 0.42359249329758714 recall 0.4297101872870815 prec: 0.5151219635918975 f1: 0.41868602949765527
  Accuracy: 0.42
  Validation took: 0:00:04

======== Epoch 2 / 8 ========
Training...

  Average training loss: 1.18
  Training epcoh took: 0:00:07

Running Validation...
acc: 0.6193029490616622 recall 0.6131894174842413 prec: 0.6330534692397215 f1: 0.6167532738581876
  Accuracy: 0.62
  Validation took: 0:00:03

======== Epoch 3 / 8 ========
Training...

  Average training loss: 0.86
  Training epcoh took: 0:00:07

Running Validation...
acc: 0.6702412868632708 recall 0.6766088437693839 prec: 0.6798091254639387 f1: 0.6730273157108877
  Accuracy: 0.67
  Validation took: 0:00:05

======== Epoch 4 / 8 ========
Training...

  Average training loss: 0.55
  Training epcoh took: 0:00:07

Running Validation...
acc: 0.7158176943699732 recall 0.7287968095920084 prec: 0.73

I0307 22:54:26.996390 140645836187392 configuration_utils.py:151] loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-config.json from cache at /home/nikhil.pinnaparaju/.cache/torch/transformers/4dad0251492946e18ac39290fcfe91b89d370fee250efe9521476438fe8ca185.8f56353af4a709bf5ff0fbc915d8f5b42bfff892cbb6ac98c3c45f481a03c685
I0307 22:54:26.999037 140645836187392 configuration_utils.py:168] Model config {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "finetuning_task": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "num_labels": 4,
  "output_attentions": false,
  "output_hidden_states": false,
  "output_past": true,
  "pruned_heads": {},
  "torchscript": false,
  "type_vocab_size": 2,
  "use_bf


======== Epoch 1 / 8 ========
Training...

  Average training loss: 1.37
  Training epcoh took: 0:00:07

Running Validation...
acc: 0.3941018766756032 recall 0.4143765238592825 prec: 0.623342803030303 f1: 0.37046635802268446
  Accuracy: 0.40
  Validation took: 0:00:04

======== Epoch 2 / 8 ========
Training...

  Average training loss: 1.18
  Training epcoh took: 0:00:07

Running Validation...
acc: 0.6085790884718498 recall 0.6055289200319626 prec: 0.6074093456914241 f1: 0.6028874122304198
  Accuracy: 0.61
  Validation took: 0:00:04

======== Epoch 3 / 8 ========
Training...

  Average training loss: 0.89
  Training epcoh took: 0:00:06

Running Validation...
acc: 0.6595174262734584 recall 0.6578262339418526 prec: 0.6675905120095436 f1: 0.6587138090447451
  Accuracy: 0.66
  Validation took: 0:00:03

======== Epoch 4 / 8 ========
Training...

  Average training loss: 0.65
  Training epcoh took: 0:00:06

Running Validation...
acc: 0.7024128686327078 recall 0.6931833548466408 prec: 0.7142

I0307 22:55:54.493494 140645836187392 configuration_utils.py:151] loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-config.json from cache at /home/nikhil.pinnaparaju/.cache/torch/transformers/4dad0251492946e18ac39290fcfe91b89d370fee250efe9521476438fe8ca185.8f56353af4a709bf5ff0fbc915d8f5b42bfff892cbb6ac98c3c45f481a03c685
I0307 22:55:54.495538 140645836187392 configuration_utils.py:168] Model config {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "finetuning_task": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "num_labels": 4,
  "output_attentions": false,
  "output_hidden_states": false,
  "output_past": true,
  "pruned_heads": {},
  "torchscript": false,
  "type_vocab_size": 2,
  "use_bf


======== Epoch 1 / 8 ========
Training...

  Average training loss: 1.37
  Training epcoh took: 0:00:07

Running Validation...
acc: 0.34316353887399464 recall 0.3509829166470771 prec: 0.4285447191697192 f1: 0.2934302241822718
  Accuracy: 0.35
  Validation took: 0:00:04

======== Epoch 2 / 8 ========
Training...

  Average training loss: 1.22
  Training epcoh took: 0:00:07

Running Validation...
acc: 0.5254691689008043 recall 0.5255840720126435 prec: 0.5271715875053488 f1: 0.5233234126696631
  Accuracy: 0.53
  Validation took: 0:00:04

======== Epoch 3 / 8 ========
Training...

  Average training loss: 0.99
  Training epcoh took: 0:00:07

Running Validation...
acc: 0.546916890080429 recall 0.5523176327061039 prec: 0.5799944052332587 f1: 0.5321564559904173
  Accuracy: 0.54
  Validation took: 0:00:03

======== Epoch 4 / 8 ========
Training...

  Average training loss: 0.74
  Training epcoh took: 0:00:07

Running Validation...
acc: 0.6702412868632708 recall 0.6652098319391553 prec: 0.6934

I0307 22:57:26.576380 140645836187392 configuration_utils.py:151] loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-config.json from cache at /home/nikhil.pinnaparaju/.cache/torch/transformers/4dad0251492946e18ac39290fcfe91b89d370fee250efe9521476438fe8ca185.8f56353af4a709bf5ff0fbc915d8f5b42bfff892cbb6ac98c3c45f481a03c685
I0307 22:57:26.579227 140645836187392 configuration_utils.py:168] Model config {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "finetuning_task": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "num_labels": 4,
  "output_attentions": false,
  "output_hidden_states": false,
  "output_past": true,
  "pruned_heads": {},
  "torchscript": false,
  "type_vocab_size": 2,
  "use_bf


======== Epoch 1 / 8 ========
Training...

  Average training loss: 1.37
  Training epcoh took: 0:00:07

Running Validation...
acc: 0.45308310991957107 recall 0.452676723335946 prec: 0.4873879217257004 f1: 0.45293906355190644
  Accuracy: 0.45
  Validation took: 0:00:04

======== Epoch 2 / 8 ========
Training...

  Average training loss: 1.15
  Training epcoh took: 0:00:07

Running Validation...
acc: 0.5576407506702413 recall 0.5560672705450832 prec: 0.5985811357473979 f1: 0.5563161955562781
  Accuracy: 0.55
  Validation took: 0:00:04

======== Epoch 3 / 8 ========
Training...

  Average training loss: 0.92
  Training epcoh took: 0:00:07

Running Validation...
acc: 0.646112600536193 recall 0.6484872115565353 prec: 0.6465364462937279 f1: 0.6361560301484187
  Accuracy: 0.65
  Validation took: 0:00:04

======== Epoch 4 / 8 ========
Training...

  Average training loss: 0.66
  Training epcoh took: 0:00:06

Running Validation...
acc: 0.6916890080428955 recall 0.6923467329747603 prec: 0.6909